## Functions as First-Class Objects

### Introduction



Functions in Python are first-class objects:


*   Created at runtime
*   Assigned to a variable or element in a data structure
*   Passed as an argument to a function
*   Returned as the result of a function
*   Examples of first-class objects in Python : `int`, `str`, `dict`


In [1]:
# Example 7-1
def factorial(n):    # Create at runtime
    """ return n! """
    return 1 if n < 2 else n * factorial(n -1)   # Returned as the result of function

print(factorial(42))
print(factorial.__doc__)
print(type(factorial))

1405006117752879898543142606244511569936384000000000
 return n! 
<class 'function'>


In [2]:
fact = factorial    # Assigned to variable
print(fact)
print(fact(5))
print(list(map(factorial, range(11))))    # Passed as an argument to a function

<function factorial at 0x7d51c83e9990>
120
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


### Higher-Order Functions



*   A function that takes a function as an arguement or returns as the result.
*   Examples : `map`, `filter`, `reduce`, `apply`, `sorted`
*   `listcomp`, `genexp` are alternatives of `map`, `filter`
*   `map`, `filter` return generators
*   `reduce` must be imported from `functools`



The apply function was deprecated in Python 2.3 and removed in Python 3 because it’s no longer necessary.  
If you need to call a function with a dynamic set of arguments, you can write fn(*args, **kwargs) instead of apply(fn, args, kwargs).

In [3]:
# Example 7-3
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
sorted(fruits, key=len)  # Any one-argument function can be used as the key

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']

In [4]:
# Example 7-4
def reverse(word):
    return word[::-1]

sorted(fruits, key=reverse)

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

In [5]:
print(list(map(factorial, range(5))))
print([factorial(n) for n in range(5)])

[1, 1, 2, 6, 24]
[1, 1, 2, 6, 24]


In [6]:
print(list(map(factorial, filter(lambda n : n % 2, range(5)))))
print([factorial(n) for n in range(5) if n % 2])

[1, 6]
[1, 6]


In [7]:
from functools import reduce
from operator import add

print(f'reduce : {reduce(add, range(100))}')
print(f'without reduce : {sum(range(100))}')

reduce : 4950
without reduce : 4950


*   Common idea of `reduce` and `sum` is to apply some operation to successive items in a series, accumulatiing previous results, thus reducing a series of values to a single value.
*   Other reducing built-in functions : `all`, `any`
*   `all(iterable)`  ---->  Returns `True` if there are no falsy elements in the iterable.  ---->  `all([])` returns `True`.
*   `any(iterable)`  ---->  Returns `True` if any element of the `iterable` is truthy.  ---->  `any([])` returns `False`.

#### Anonymous Functions



*   Created by `lambda` keyword in a Python expression
*   The body can not contain other python statements such as `while`, `try`, `=`
*   `:=` can be used (better to replace `lambda` function with regular function).
*   Useful in the context of an argument list for a higher-order function.



In [8]:
sorted(fruits, key=lambda word : word[::-1])

['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']

#### Nine Flavors of Callable Objects



*   Use `callable()` to determine whether an object is callable.
*   Callable types:
  *   User-defined functions : Created with `def`, `lambda`
  *   Built-in functions : `len`, `time.strftime`, `type`
  *   Built-in methods : `dict.get`, `list.append`
  *   Methods : Functions defined in the body of a class
  *   Classes : `__new__` method create an instance, `__init__` initialize it. Calling a class is like calling function.
  *   Class instances : if class has `__call__`
  *   Generator functions : Functions use `yield` in body
  *   Native coroutine functions : Functions defined with `async def`, return a coroutine object.
  *   Asynchronous generator functions : Functions defined with `async def`, have `yield` in body, return asynchronous generator, use with `async for`.

*   Native coroutine and asynchronous generator functions return objects that only work with the help of an asynchronous programming framework like `asyncio`.



#### User-Defined Callable Types



*   Arbitrary Python objects can behave like functions by implementing `__call__` instance method.
*   Another good use case for `__call__` is implementing decorators.



In [9]:
# Example 7-8
import random

class BingoCage:

    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)

    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')

    def __call__(self):    # Shortcut to bingo.pick() : bingo()
        return self.pick()


bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())

1
2


### From Positional to Keyword-Only Parameters



*   `*`, `**` to unpack iterables and mappings into seperate arguments when calling a function
*   To specify keyword-only arguments when defining a function, name them after the argument prefixed with `*`.
*   Keyword-only arguments do not need to have a default value.
*   If don't want to support variable positional arguments but still want keyword-only arguments, put `*` by itself in the signature.



In [10]:
# Example 7-9
def tag(name, *content, class_=None, **attrs):
    """Generate one or more HTML tags"""
    if class_ is not None:
        attrs['class'] = class_
    attr_pairs = (f' {attr}="{value}"' for attr, value
                    in sorted(attrs.items()))
    attr_str = ''.join(attr_pairs)
    if content:
        elements = (f'<{name}{attr_str}>{c}</{name}>'
                    for c in content)
        return '\n'.join(elements)
    else:
        return f'<{name}{attr_str} />'

In [11]:
print(tag('br'))
print('-----------------------------')
print(tag('p', 'hello'))  # Any number of arguments after the first are captured by *content as a tuple
print('-----------------------------')
print(tag('p', 'hello', 'world'))
print('-----------------------------')
print(tag('p', 'hello', id=33))  # Keyword arguments not explicitly named in tag signature are captured by **attrs as dict
print('-----------------------------')
print(tag('p', 'hello', 'world', class_='sidebar'))  # class_ paramter can only be passed as a keyword argument
print('-----------------------------')
print(tag(content='testing', name='img'))  # first positional argument can also be passed as a keyword
print('-----------------------------')
my_tag = {'name' : 'img', 'title': 'Sunset Boulevard',
          'src' : 'sunset.jpg', 'class' : 'framed'
}

print(tag(**my_tag))  # passes all my_tag items as seperate arguments which are then bound to named parameters

<br />
-----------------------------
<p>hello</p>
-----------------------------
<p>hello</p>
<p>world</p>
-----------------------------
<p id="33">hello</p>
-----------------------------
<p class="sidebar">hello</p>
<p class="sidebar">world</p>
-----------------------------
<img content="testing" />
-----------------------------
<img class="framed" src="sunset.jpg" title="Sunset Boulevard" />


#### Positional-Only Parameters



*   To define a function requiring positional-only paramters, use `/` in parameter list.
*   All arguments to the left of the `/` are positional-only. After `/` other arguments work as usual.
*   Example : built-in `divmod` can only be called with positional parameters


In [12]:
# divmod(a=10, b=20)  ----> TypeError

def divmod(a, b, /):
    return (a // b, a % b)

### Packages for Functional Programming



*   Two useful packages : `operator`, `functools`
*   `operator` : Provides function equivalents for dozens of operators.
*   `functools` : Provides higher-order functions


#### The operator Module



*   Arithmetic operators : `add`, `mul`
*   `itemgetter` : Creates a function that given a collection, returns the item at index `i`, supports any class that implements `__getitem__`.

*   `attrgetter` : Creates a function to extract object attributes by name, can navigate through nested objects if contains `.` .
*   `methodcaller` : It's similar to `itemgetter` and `attrgetter` in that it creates a function on the fly. The function it creates calls a method by name on the object given as argument.



In [13]:
# Example 7-11 , 7-12
from functools import reduce
from operator import mul


def factorial_no_operator(n):
    return reduce(lambda a, b : a * b, range(1, n + 1))

def factorial_with_operator(n):
    return reduce(mul, range(1, n + 1))

In [14]:
# Example 7-13
from operator import itemgetter


metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

for city in sorted(metro_data, key=itemgetter(1)):  # sort by index 1 which is country code
    print(city)

print('\n-----------------------------------\n')

cc_name = itemgetter(1, 0)  # passing multiple index creates a function that return tuple with extracted values
for city in metro_data:
    print(cc_name(city))

('São Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))

-----------------------------------

('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'São Paulo')


In [15]:
# Example 7-14
from collections import namedtuple
from operator import attrgetter


LatLon = namedtuple('LatLon', 'lat lon')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')

metro_areas = [Metropolis(name, cc, pop, LatLon(lat, lon)) for name, cc, pop, (lat, lon) in metro_data]

print(metro_areas[0])
print(metro_areas[0].coord.lat)

print('\n-----------------------------------\n')

name_lat = attrgetter('name', 'coord.lat')  # retrieve name and coord.lat nested attribute
for city in sorted(metro_areas, key=attrgetter('coord.lat')):  # sort list of cities by latitude
    print(name_lat(city))  # show only name and lat

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLon(lat=35.689722, lon=139.691667))
35.689722

-----------------------------------

('São Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [16]:
# Example 7-15
from operator import methodcaller


s = 'The time has come'
upcase = methodcaller('upper')
print(upcase(s))

hyphenate = methodcaller('replace', ' ', '-')
print(hyphenate(s))

THE TIME HAS COME
The-time-has-come


#### Freezing Arguments with functools.partial



*   `partial` : given a callable it produces a new callable with some of the arguments the original callable bound to predetermined values. It takes a callable as first argument, followed by an arbitrary number of positional and keyword arguments to bind, returns `functools.partial` object, has attributes providing access to original function and fixed arguments.
*   `partialmethod` : same as `partial` but is designed to work with methods.



In [17]:
# Example 7-16
from functools import partial


triple = partial(mul, 3)  # binding the first positional argument of mul to 3
list(map(triple, range(1, 10)))

[3, 6, 9, 12, 15, 18, 21, 24, 27]

In [18]:
# Example 7-17
import unicodedata

nfc = partial(unicodedata.normalize, 'NFC')

s1 = 'café'
s2 = 'cafe\u0301'

print(s1, s2)

print(s1 == s2)

print(nfc(s1) == nfc(s2))

café café
False
True


In [19]:
# Example 7-18
picture = partial(tag, 'img', class_='pic-frame')
print(picture(src='wumpus.jpg'))
print('\n-------------------------------------\n')
print(picture)
print('\n-------------------------------------\n')
print(f'dir picture : {dir(picture)}')
print('\n-------------------------------------\n')
print(f'function : {picture.func}')
print('\n-------------------------------------\n')
print(f'keyword arg : {picture.keywords}')
print('\n-------------------------------------\n')
print(f'args : {picture.args}')

<img class="pic-frame" src="wumpus.jpg" />

-------------------------------------

functools.partial(<function tag at 0x7d51c83ea050>, 'img', class_='pic-frame')

-------------------------------------

dir picture : ['__call__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__vectorcalloffset__', 'args', 'func', 'keywords']

-------------------------------------

function : <function tag at 0x7d51c83ea050>

-------------------------------------

keyword arg : {'class_': 'pic-frame'}

-------------------------------------

args : ('img',)


### Lecturers



*   Reihane Heidari : [Linkedin](https://www.linkedin.com/in/reihane-heidari/)
*   Saeed Hemmati : [Linkedin](https://www.linkedin.com/in/saeed-hemati/)


Presentation Date : 11-03-2023


### Reviewers

1. Hosein Toodehroosta, review date: 11-02-2023, [LinkedIn](https://www.linkedin.com/in/hossein-toodehroosta-4b34b9191?utm_source=share&utm_campaign=share_via&utm_content=profile&utm_medium=android_app)
2. Mahya Asgarian, review date : 11-02-2023, [Linkedin](https://www.linkedin.com/in/mahya-asgarian-9a7b13249/)